In [3]:
import matplotlib
matplotlib.use("Agg")
import xgboost as xgb
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import torch.nn as nn
from models import Decoder, Embedder, StrongDecoder
from train2 import initialize_embedder, train_surrogate, train_xgb, improve_embedder, freeze
from utils import standardize, load_data_custom, get_z_features
from analysis import final_embedding_analysis, representation_report, plot_embedding, plot_reconstruction
from config import Z_DIM, D_HIDDEN, DEVICE

NUM_LOOPS = 8
EPOCHS_SURROGATE = 8
EPOCHS_AE = 5
EPOCHS_IMPROVE = 10

/Users/jaspreet.brar/ml-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
fpath = 'data/tejas.csv'
print(f"Loading data from {fpath}")
raw_X, raw_y = load_data_custom(fpath, 'TARGET')
raw_X_train, raw_X_test, y_train, y_test = train_test_split(raw_X, raw_y, random_state=42, test_size=0.2, shuffle=True, stratify=raw_y)
std_X_train = standardize(raw_X_train)
std_X_test = standardize(raw_X_test, raw_X_train)
X_train_np = std_X_train.values
X_test_np = std_X_test.values
input_dim = X_train_np.shape[1]
# train xgb on raw X
xgb_teacher = train_xgb(X_train_np, y_train, X_test_np, y_test)
dtest = xgb.DMatrix(X_test_np, nthread=8)
teacher_preds = xgb_teacher.predict(dtest)
teacher_auc = roc_auc_score(y_test, teacher_preds)
print(f" Teacher (XGBoost) Test AUC: {teacher_auc:.4f}")
# initialize models
embedder = initialize_embedder(X_train_np, input_dim)
ae_embedder = initialize_embedder(X_train_np, input_dim)
Z_ae_test = get_z_features(ae_embedder, X_test_np)
# we make this to ensure embedder doesnt give garbage
decoder = Decoder(Z_DIM, D_HIDDEN, input_dim).to(DEVICE)
# training
print("\n" + "="*50)
print(f"🔄 STARTING TRAINING LOOP ({NUM_LOOPS} Cycles)")
print("="*50)
surrogate = None
Z_prev_train = X_train_np
Z_prev_test = X_test_np

Loading data from data/tejas.csv
 Teacher (XGBoost) Test AUC: 0.7485

🔄 STARTING TRAINING LOOP (8 Cycles)


In [5]:
for i in range(NUM_LOOPS):
    print(f"\n--- Cycle {i+1} / {NUM_LOOPS} ---")
    if surrogate is not None:
        freeze(surrogate)
    # 1. Current embeddings
    Z_curr_train = get_z_features(embedder, X_train_np)
    Z_curr_test  = get_z_features(embedder, X_test_np)
    # 2. Teacher input = [Z_{k-1} | Z_k]
    if i < 3:
      X_train_xgb = np.hstack([Z_prev_train, Z_curr_train])
      X_test_xgb  = np.hstack([Z_prev_test,  Z_curr_test])
    else :
      X_train_xgb = Z_curr_train
      X_test_xgb  = Z_curr_test
    # 3. TRAIN XGBOOST (On Augmented Data)al
    print(f"     Training XGBoost on {X_train_xgb.shape[1]} features...")
    xgb_teacher = train_xgb(X_train_xgb, y_train, X_test_xgb, y_test)
    dtest = xgb.DMatrix(X_test_xgb, nthread=8)
    xgb_auc = roc_auc_score(y_test, xgb_teacher.predict(dtest))
    # print(f"     XGBoost AUC: {xgb_auc:.4f}")
    # 4. Teacher labels (FULL VIEW)
    teacher_labels = xgb_teacher.predict(xgb.DMatrix(X_train_xgb))
    # 5. TRAIN SURROGATE
    # Input: Z_{k} | Target: New XGB Labels
    surrogate = train_surrogate(Z_curr_train, teacher_labels)
    # 6. IMPROVE EMBEDDER
    # Input: Raw Data | Target: New XGB Labels
    embedder = improve_embedder(X_train_np, y_train, embedder, surrogate)
    # 7. Store Zk for next round
    Z_prev_train = Z_curr_train.copy()
    Z_prev_test  = Z_curr_test.copy()
    # 8. stats
    representation_report(
        cycle=i+1,
        embedder=embedder,
        surrogate=surrogate,
        xgb_teacher=xgb_teacher,
        Z_prev_train=Z_prev_train,
        Z_curr_train=Z_curr_train,
        Z_curr_test=Z_curr_test,
        X_test_xgb=X_test_xgb,
        y_train=y_train,
        y_test=y_test,
        device=DEVICE
    )


--- Cycle 1 / 8 ---
     Training XGBoost on 185 features...

📊 REPRESENTATION REPORT — Cycle 1
Teacher AUC          : 0.7488
Teacher MSE          : 0.2003
Student AUC          : 0.7296
Student MSE          : 0.2027
Δ MSE (Student−Teacher): +0.0024
Z Linear Probe AUC   : 0.7080
Z Drift              : 0.000000
Centroid Distance    : 0.6709

--- Cycle 2 / 8 ---
     Training XGBoost on 128 features...

📊 REPRESENTATION REPORT — Cycle 2
Teacher AUC          : 0.7421
Teacher MSE          : 0.2023
Student AUC          : 0.7404
Student MSE          : 0.2090
Δ MSE (Student−Teacher): +0.0067
Z Linear Probe AUC   : 0.7408
Z Drift              : 0.000000
Centroid Distance    : 1.0890

--- Cycle 3 / 8 ---
     Training XGBoost on 128 features...

📊 REPRESENTATION REPORT — Cycle 3
Teacher AUC          : 0.7452
Teacher MSE          : 0.2016
Student AUC          : 0.7419
Student MSE          : 0.2023
Δ MSE (Student−Teacher): +0.0007
Z Linear Probe AUC   : 0.7434
Z Drift              : 0.000000
Cent

In [6]:
print("\n" + "="*80)
print(f"{'TRAINING PROGRESS REPORT':^80}")
print("="*80)
# --- 6. ANALYSIS ---
final_embedding_analysis(
embedder=embedder,
decoder=decoder,
X_train=X_train_np,
X_test=X_test_np,
y_train=y_train,
y_test=y_test,
device=DEVICE)


                            TRAINING PROGRESS REPORT                            

🔍 FINAL EMBEDDING ANALYSIS
📈 Z Linear Probe AUC: 0.7427


/Users/jaspreet.brar/ml-env/lib/python3.11/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


📐 Centroid distance (fraud vs legit): 1.0596
🔁 Reconstruction MSE: 3.1369
🔑 Most important Z dims (linear probe): [10, 9, 60, 59, 61, 6, 5, 31, 29, 36]
✔ Interpretation guide:
- Clear class separation → Z learned structure
- High probe AUC → Z encodes truth
- Low recon error → Z preserved info
- Tight clusters → stable embedding


In [7]:
vec = X_test_np[0]
z = embedder(torch.tensor(vec, device=DEVICE)).cpu().numpy()
p = torch.randn(1, input_dim, device=DEVICE, requires_grad=True)
optimizer = torch.optim.Adam([p], lr=1e-2)
loss_fn = nn.MSELoss()
z_target = torch.tensor(z, device=DEVICE).unsqueeze(0)
for _ in range(500):
    optimizer.zero_grad()
    z_p = embedder(p)
    loss = loss_fn(z_p, z_target)
    loss.backward()
    optimizer.step()
p_recon = p.detach().cpu().numpy()
print("og_vec : ", vec)
print("recon_vec : ", p_recon)

og_vec :  [ 1.15336883e+00 -3.24585676e-01  1.38717163e+00  1.39386368e+00
  6.64271057e-01  8.11107457e-01 -4.38414514e-02  4.24492717e-01
  2.45743483e-01  4.05436456e-01 -2.37868142e+00  9.15270090e-01
 -1.68622971e+00 -4.04029608e-01 -3.04918438e-01 -7.74242342e-01
  5.78408539e-01 -4.51414317e-01  5.99675059e-01 -9.33419228e-01
  5.82867742e-01  2.01031705e-03  4.67698932e-01 -4.99500692e-01
  4.33759466e-02 -6.25066876e-01 -2.45247349e-01 -5.06825626e-01
  9.33616579e-01 -1.02782324e-01 -6.25997484e-02 -9.72736120e-01
  1.51295209e+00 -1.23486318e-01 -2.32186317e-01 -2.07223311e-01
 -2.90647894e-01 -5.48148513e-01 -4.69092488e-01  1.25078750e+00
  0.00000000e+00  4.18695748e-01 -4.27240670e-01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+0

In [8]:
mse = np.mean((vec - p_recon)**2)
mse

2.1292255

In [9]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 3))
plt.scatter(vec, p_recon, s=5, alpha=0.6)
plt.xlabel("Original")
plt.ylabel("Reconstructed")
plt.tight_layout()
plt.savefig(
    f"plots/recons.png",
    dpi=1000,
    bbox_inches="tight"
)
plt.close()

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
def linear_probe_auc(Z, y):
    clf = LogisticRegression(max_iter=3000)
    clf.fit(Z, y)
    return roc_auc_score(y, clf.predict_proba(Z)[:, 1])
def centroid_distance(Z, y):
    fraud = Z[y == 1]
    legit = Z[y == 0]
    if len(fraud) == 0 or len(legit) == 0:
        return 0.0
    return np.linalg.norm(fraud.mean(0) - legit.mean(0))
# ============================================================
# FINAL EMBEDDING COMPARISON (CORRECT & FAIR)
# ============================================================
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
os.makedirs("plots", exist_ok=True)
# -------------------------
# Helper metrics
# -------------------------
def linear_probe_auc(Z, y):
    clf = LogisticRegression(max_iter=3000)
    clf.fit(Z, y)
    return roc_auc_score(y, clf.predict_proba(Z)[:, 1])
def centroid_distance(Z, y):
    f = Z[y == 1]
    l = Z[y == 0]
    return np.linalg.norm(f.mean(0) - l.mean(0))
# -------------------------
# Helper plots
# -------------------------
def scatter_plot(Z2, y, name):
    plt.figure(figsize=(4,3))
    plt.scatter(Z2[:,0], Z2[:,1], c=y, s=6, cmap="coolwarm")
    plt.tight_layout()
    plt.savefig(f"plots/{name}.png", dpi=150)
    plt.close()
# ============================================================
# 1️⃣ YOUR EMBEDDING (with post-hoc decoder)
# ============================================================
Z_yours = get_z_features(embedder, X_test_np)
print("\nYOUR EMBEDDING")
print("AUC:", linear_probe_auc(Z_yours, y_test))
print("Centroid:", centroid_distance(Z_yours, y_test))
# ---- Train decoder ONLY for your embedding
# task_decoder = Decoder(Z_DIM, D_HIDDEN, input_dim).to(DEVICE)
task_decoder = StrongDecoder(
z_dim=Z_DIM,
hidden_dims=[Z_DIM, (Z_DIM + X_test_np.shape[1])//2, X_test_np.shape[1]],
out_dim=input_dim
).to(DEVICE)
for p in embedder.parameters(): p.requires_grad = False
opt = optim.Adam(task_decoder.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
X_train_t = torch.tensor(X_train_np, device=DEVICE)
for _ in range(5):
    for i in range(0, len(X_train_t), 2048):
        xb = X_train_t[i:i+2048]
        with torch.no_grad():
            z = embedder(xb)
        loss = loss_fn(task_decoder(z), xb)
        opt.zero_grad()
        loss.backward()
        opt.step()
with torch.no_grad():
    X_hat = task_decoder(torch.tensor(Z_yours, device=DEVICE)).cpu().numpy()
print("Recon MSE:", mean_squared_error(X_test_np, X_hat))
# PCA projection plot
scatter_plot(PCA(2).fit_transform(Z_yours), y_test, "yours_pca")
# Reconstruction scatter
f = np.random.randint(0, X_test_np.shape[1])
plt.figure(figsize=(4,3))
plt.scatter(X_test_np[:,f], X_hat[:,f], s=5)
plt.tight_layout()
plt.savefig("plots/yours_recon.png", dpi=150)
plt.close()
# ============================================================
# 3️⃣ AUTOENCODER BASELINE (own encoder + own decoder)
# ============================================================
ae_embedder = Embedder(input_dim, D_HIDDEN, Z_DIM).to(DEVICE)
ae_decoder  = Decoder(Z_DIM, D_HIDDEN, input_dim).to(DEVICE)
opt = optim.Adam(list(ae_embedder.parameters()) + list(ae_decoder.parameters()), lr=1e-3)
loss_fn = nn.MSELoss()
X_train_t = torch.tensor(X_train_np, device=DEVICE)
for _ in range(5):
    for i in range(0, len(X_train_t), 2048):
        xb = X_train_t[i:i+2048]
        z = ae_embedder(xb)
        loss = loss_fn(ae_decoder(z), xb)
        opt.zero_grad()
        loss.backward()
        opt.step()
Z_ae = ae_embedder(torch.tensor(X_test_np, device=DEVICE)).detach().cpu().numpy()
X_hat_ae = ae_decoder(torch.tensor(Z_ae, device=DEVICE)).detach().cpu().numpy()
print("\nAUTOENCODER BASELINE")
print("AUC:", linear_probe_auc(Z_ae, y_test))
print("Centroid:", centroid_distance(Z_ae, y_test))
print("Recon MSE:", mean_squared_error(X_test_np, X_hat_ae))
scatter_plot(PCA(2).fit_transform(Z_ae), y_test, "ae_pca")
f = np.random.randint(0, X_test_np.shape[1])
plt.figure(figsize=(4,3))
plt.scatter(X_test_np[:,f], X_hat_ae[:,f], s=5)
plt.tight_layout()
plt.savefig("plots/ae_recon.png", dpi=150)
plt.close()


YOUR EMBEDDING
AUC: 0.7443782447274898
Centroid: 1.0697653
Recon MSE: 0.7874517440795898

AUTOENCODER BASELINE
AUC: 0.7178601799137526
Centroid: 0.76531863
Recon MSE: 0.35202455520629883


In [ ]:
class StrongerDecoder(nn.Module):
    def __init__(self, z_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.GELU(),
            nn.Linear(256, 256),
            nn.GELU(),
            nn.Linear(256, out_dim)
        )

    def forward(self, z):
        return self.net(z)


def xgb_auc(Z_train, Z_test, y_train, y_test):
    dtrain = xgb.DMatrix(Z_train, label=y_train)
    dtest  = xgb.DMatrix(Z_test, label=y_test)

    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "max_depth": 5,
        "eta": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "nthread": 8
    }

    model = xgb.train(params, dtrain, num_boost_round=300)
    preds = model.predict(dtest)
    return roc_auc_score(y_test, preds)


def recon_eval_fast(Z_train, Z_test, X_train, X_test):
    # closed-form linear regression
    W, _, _, _ = np.linalg.lstsq(Z_train, X_train, rcond=None)
    X_hat = Z_test @ W

    errs = ((X_hat - X_test) ** 2).mean(axis=1)

    plt.hist(errs, bins=100)
    plt.yscale("log")
    plt.title("Linear Reconstruction MSE")
    plt.show()

    return {
        "mse_mean": errs.mean(),
        "mse_p50": np.percentile(errs, 50),
        "mse_p90": np.percentile(errs, 90),
    }

def evaluate_embedding(name, Z_train, Z_test):
    print(f"\n===== {name} =====")

    auc = xgb_auc(Z_train, Z_test, y_train, y_test)
    print("XGB AUC:", round(auc, 4))

    out = {"auc": auc}

    if Z_train.shape[1] >= 2:
        recon = recon_eval_fast(
            Z_train, Z_test,
            X_train_np, X_test_np
        )
        print(
            "Recon (linear) MSE:",
            "mean", round(recon["mse_mean"], 4),
            "| p50", round(recon["mse_p50"], 4),
            "| p90", round(recon["mse_p90"], 4),
        )
        out.update(recon)
    else:
        print("Recon skipped")

    return out
